In [1]:
data read

SyntaxError: invalid syntax (<ipython-input-1-b0f673b73b7d>, line 1)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
# recreating: https://www.kaggle.com/apapiu/house-prices-advanced-regression-techniques/regularized-linear-models/notebook
# (not forking, as the goal is to use it as a tutorial)
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr
%matplotlib inline

In [ ]:
train = pd.read_csv("../input/train.csv")
test  = pd.read_csv("../input/test.csv")

print(train.shape)
print(test.shape)
# contains 'SalePrice', which not in test
train.head()

data load

In [ ]:
# start and stop features. everything except for 'id' and 'saleprice'
feat_s = 'MSSubClass'
feat_e = 'SaleCondition'
# http://pandas.pydata.org/pandas-docs/stable/merging.html 
# two options. note that 'train' has one extra column 'SalePrice' which must be omitted
# creating a tuple of dataframes
df_total1 = pd.concat((train.loc[:,feat_s:feat_e], test.loc[:,feat_s:feat_e]))
print(df_total1.shape)
# or just taking the whole thing and dropping what we don't want
df_total2 = pd.concat([train.drop(['SalePrice'],axis=1),test])
df_total2.drop(['Id'], axis=1, inplace=True)
print(df_total2.shape)
# compare the two to verify that this worked correctly
#print(np.alltrue(df_total1.columns == df_total.columns))
#print(np.alltrue(df_total1.index == df_total.index))
# print(df_total2.head())
# print(df_total1.head())
# not "equal", but likely just because of NaN. regardless, not taking any risks.
# df_total2 == df_total1
# use original approach
alldf = df_total1

data preproc

kaggle deleted my progress, going to re-create really fast and without documentation

- normalise: visualisation
- normalise: xfrm skewed num feats with log(feat + 1)
- dummy vars
- s/NaN/mean()/

In [ ]:
# normalise - visualisation
bigimg = 0
if(bigimg == 1):
    matplotlib.rcParams['figure.figsize'] = (12.0,6.0)
# tmp df for visualisation
prices = pd.DataFrame({"price":train["SalePrice"], "log(price + 1)":np.log1p(train["SalePrice"])})

In [ ]:
# log transform target
# TODODOC: np.log1p
train["SalePrice"] = np.log1p(train["SalePrice"])

# log transform skewed numeric features
# generate a selector # TODO: does it have to be with '!=object' or can that one pd data type be used?
num_feats = alldf.dtypes[alldf.dtypes != "object"].index

# compute skew
# generate selector:
# TODODOC; doc for scipi.stats.skew (np and wiki)
skewed_feats = train[num_feats].apply(lambda x: skew(x.dropna()))
skewed_feats = skewed_feats[skewed_feats > 0.75] # TODO: why?
skew_idx = skewed_feats.index

alldf[skew_idx] = np.log1p(alldf[skew_idx])

In [ ]:
# get_dummies - essentially: convert non-numeric features to binary series
# expand features with non-numeric into multiple columns
# TODO:  e.g. SaleCondition
# original values: # note: this is the wrong way to "uniqify"!# pd.get_dummies(alldf['SaleCondition']).columns
# Index(['Abnorml', 'AdjLand', 'Alloca', 'Family', 'Normal', 'Partial'], dtype='object')
# after get_dummies on entire df:
# ['SaleCondition_Abnorml', 'SaleCondition_AdjLand','SaleCondition_Alloca', 'SaleCondition_Family', 'SaleCondition_Normal','SaleCondition_Partial']
alldf = pd.get_dummies(alldf)
print("after get_dummies: ",alldf.columns)

In [ ]:
# - s/NaN/mean()/
tmpdf = alldf.fillna(alldf.mean())
alldf.fillna(alldf.mean(),inplace=True)
# verify that 'inplace' works:
np.alltrue(tmpdf == alldf)

Models
- create matrices for sklearn
- regularizations:
    - l1_lasso
    - l2_ridge
- create function for calc of RMSE

TODODOC:

add all the links about sklearn model_selection and cross_val_score

basically: for cross_val_score, higher scores are, like, the best,
so the retval of neg_mean_squared_error is negative because it's, like, the worst, and

In [ ]:
# create matrices for sklearn
# TODO: learn and understand the selector syntax. I think this is selecting all until 1460 and all from 1460
print(train.shape)
X_train = alldf[:train.shape[0]] # 1460
X_test  = alldf[train.shape[0]:]
y = train.SalePrice

In [ ]:
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score

# TODODOC
def rmse_cv(model):
    rmse = np.sqrt(-cross_val_score(model, X_train, y, scoring="neg_mean_squared_error", cv = 5))
    return rmse


In [ ]:
# L2 Ridge
model_ridge = Ridge()

alphas = [0.05, 0.1, 0.3, 1, 3, 5, 7,8,9, 10, 11, 12, 15, 30, 50, 75]
#alphas = [0.05, 0.1, 0.3, 1, 3, 5, 10, 15, 30, 50, 75]
cv_ridge = [rmse_cv(Ridge(alpha = alpha)).mean() for alpha in alphas]

cv_ridge = pd.Series(cv_ridge, index = alphas)
cv_ridge.plot(title = "Validation - Just Dü It")
plt.xlabel("alpha")
plt.ylabel("rmse")

# random
model_ridge = RidgeCV(alphas = alphas).fit(X_train,y)

In [ ]:
# get min value
# ref: 0.1273373466867076
cv_ridge.min()

In [ ]:
# L1 Lasso
# Lasso does feature selection - setting coefficients of features it deems unimportant to zero
alphas_l1 = [1, 0.1, 0.001, 0.0005]
model_lasso = LassoCV(alphas = alphas_l1).fit(X_train, y)
# ref: 0.12314421090977427
print(rmse_cv(model_lasso).mean())
coef = pd.Series(model_lasso.coef_, index = X_train.columns)
print("Lasso picked %s variables and eliminated the other %s variables" % (sum(coef != 0),sum(coef == 0)))
# TODO:
# Good job Lasso. One thing to note here however is that the features selected are not necessarily the "correct" ones 
# - especially since there are a lot of collinear features in this dataset. 
# One idea to try here is run Lasso a few times on boostrapped samples and see how stable the feature selection is.
print("Important Coefficients")
# matplotlib.rcParams
imp_coef = pd.concat([coef.sort_values().head(10), coef.sort_values().tail(10)])
imp_coef.plot(kind="barh")
plt.title("Coefficients in the Lasso Model")
# NOTE: most imp pos: GrLivArea 

In [ ]:
# residuals:
matplotlib.rcParams['figure.figsize'] = (6.0,6.0) # TODO: determine how to have the dots with a black border
preds = pd.DataFrame({"preds":model_lasso.predict(X_train), "true":y})
preds["residuals"] = preds["true"] - preds["preds"]
preds.plot(x = "preds", y = "residuals", kind = "scatter")

XGBOOST

In [2]:
# TODO - isntall on windows, somehow
#import xgboost as xgb
# ...

Predict On Test Set

In [ ]:
# convert back to non-log
lasso_preds = np.expm1(model_lasso.predict(X_test))
# TODO: use xgb instead of ridge
ridge_preds = np.expm1(model_ridge.predict(X_test))

In [ ]:
predictions = pd.DataFrame({"ridge":ridge_preds,"lasso":lasso_preds})
predictions.plot(x = "ridge", y = "lasso", kind = "scatter")

# predictions = pd.DataFrame({"xgb":xgb_preds,"lasso":lasso_preds})
# predictions.plot(x = "xgb", y = "lasso", kind = "scatter")